In [8]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
# data_path = Path('data')
config = pd.read_excel('data/config.xlsx', sheet_name='layers')
config.head(3)

,Name,shape_type,color,alpha,label,file_type,shp,gdb,layer
0,Proposed Pipeline,NaN,orange,NaN,label,gdf,NaN,NaN,NaN
1,District Boundary,NaN,black,NaN,WDNAME,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN
2,AEWSD North Canal,NaN,blue,NaN,CANAL,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN


In [12]:
def get_layer(row):
	if row['file_type'] == "shp":
		gdf = gpd.read_file(Path(row["shp"]))
		
	elif row['file_type'] == "gdb":
		gdb = Path(row["gdb"])
		gdf = gpd.read_file(gdb,layer=row["layer"])
	else:
		# gdf = row["gdf"]
		return None
	# gpd.GeoDataFrame(
	# 		# row["gdf"],geometry=row["gdf"].geometry
	# 		)
	
	gdf["color"] = row["color"]
	# gdf["label"] = gdf[row["label"]].astype(str)
	gdf["label"] = gdf[row["label"]].astype(str)


	gdf["layer"] = row['Name']
	print(row['Name'])
	print(gdf.crs)
	
	
	return gdf[['color','label',"layer","geometry"]]#.to_crs(epsg=4326)

# Create gdb
gdfs = {y['Name']:get_layer(y) for i,y in config.iterrows()}
# gdfs = {y['Name']:y for i,y in config.iterrows()}
# gdfs['Proposed Pipeline']

District Boundary
EPSG:2229
AEWSD North Canal
EPSG:2229
FFPPP Discharge Pipeline
EPSG:2229
Frick Unit Service Area
EPSG:2229
Proposed Turnout
EPSG:6424
DiGiorgio Pipeline
EPSG:2229
DiGiorgio Service Area
EPSG:2229
Existing Laterals
EPSG:2229
Frick Unit Service Area
EPSG:2229
Panama Unit Service Area
EPSG:2229
Groundwater Service Area
EPSG:2229
Landowner Wells
EPSG:2229
Sandrini Unit proposed pipeline
EPSG:2229
Sandrini Unit Service Area
EPSG:2229


In [14]:
epsg = 4326
# epsg = 26745
# epsg = 2229
# epsg = 6424
gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])
gdf.to_parquet("data\\gdf-2023-08-15.parquet")